# Exercise 1.2
## This *.ipynb file contains:



In [2]:
import numpy as np
import torch
from Exercise1_1 import LQRSolver

In [3]:
H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.double)
M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.double)
sigma = torch.tensor([[[0.5249],[0.4072]]], dtype=torch.double) 
C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive definite
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
T = torch.tensor(1.0, dtype=torch.double)
method = 'rk4'

In [6]:
solver = LQRSolver(H, M, sigma, C, D, R, T, method)

In [8]:
N = 5000
t0 = torch.tensor(0.1,dtype = torch.double)

In [10]:
#tau = torch.clone((T-t0)/N)

In [12]:
time_grid_for_MC = torch.linspace(t0,T,N,dtype = torch.double)

In [14]:
X0 = 0.5*torch.ones([1,1,2], dtype=torch.double)
dt_for_MC = time_grid_for_MC[1:]-time_grid_for_MC[:-1]
S = solver.solve_riccati_ode(time_grid_for_MC.unsqueeze(0)).squeeze()
multpX = - M@torch.linalg.inv(D)@M.T
multpalp = -torch.linalg.inv(D)@M.T@S

In [16]:
print(solver.value_function(t0.unsqueeze(0),X0))

tensor(0.8088, dtype=torch.float64)


In [18]:
_int = torch.einsum('bii->b',sigma.squeeze(1)@sigma.squeeze(0).T@S)

value_for_varif = (X0@S[0]@X0.transpose(1,2) + torch.tensor(0.5)*dt_for_MC@(_int[1:]+_int[:-1]).unsqueeze(1)).squeeze()

print(value_for_varif)

tensor(0.8088, dtype=torch.float64)


In [20]:
J_list = torch.tensor([],dtype = torch.double)

for iii in range(500):
    J_sample = torch.tensor([],dtype = torch.double)
    for ii in range(10):
        X_0_N = X0
        for i in range(N-1):
            X_next = ((torch.eye(2)+ dt_for_MC[i]*(H + multpX@S[i]))@ X_0_N[i:].transpose(1,2)+sigma*torch.sqrt(dt_for_MC[i])*torch.randn(1)).transpose(1,2)
            X_0_N = torch.cat((X_0_N, X_next), dim=0)

        alp = multpalp@X_0_N.transpose(1,2)
        int_ = alp.transpose(1,2)@D@alp + X_0_N@C@X_0_N.transpose(1,2)
        J = X_0_N[-1]@R@X_0_N[-1].T + torch.tensor(0.5)*dt_for_MC@((int_.squeeze(1)[1:]+int_.squeeze(1)[:-1]))

        J_sample = torch.cat((J_sample, J), dim = 0)

    J_list = torch.cat((J_list,torch.mean(J_sample).unsqueeze(0)), dim = 0)

In [30]:
print(torch.sort(J_list)[0][:20])

tensor([0.6933, 0.7322, 0.7548, 0.7564, 0.7906, 0.7995, 0.8108, 0.8267, 0.8486,
        0.8515, 0.8546, 0.8587, 0.8632, 0.8795, 0.8913, 0.9102, 0.9129, 0.9173,
        0.9244, 0.9376], dtype=torch.float64)
